# Lecture: Witness Sets

In [46]:
using HomotopyContinuation

So far, we've seen that numerical methods are very good at solving **zero-dimensional polynomial systems**

**Question**: What do we do if we are interested in a **positive-dimensional** set of solutions?

**Answer**: Slice it with enough hyperplanes until we have a zero-dimensional solution set.



<img src="SimpleWitness.png" width="300">

The data in such a *slice* is called a **witness set**.

A witness set for an irreducible variety $X \subset \mathbb{C}^n$ is a triple 
$$W=(F,L,S)$$
where 
    
- $\color{blue}{F}$ is a polynomial system for which $X$ is a component, 
- $\color{green}{L}$ is a generic affine linear space of complementary dimension
- $\color{red}{S} = X \cap L$



<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
How is this a good representation of the variety $X$?
</p>


Given a witness set $W=(F,L,S)$ for $X$...

Certain information about $X$ become obvious:
- dimension (equals the codimension of $L$)
- degree (equals the number of points in $S$)

Certain algorithms become almost trivial

- Membership: is $p$ contained in $X$?

- Sampling: find $100000$ points on $X$

- Find a (pseudo) witness set for a projection of $X$

One major advantage (compared to symbolic representations) is that they can be computed fast. 

One major disadvantage is that it can be difficult to prove things (but not impossible)

<hr style="border:1px solid gray"> </hr>

## Constructing a witness set using homotopycontinuation.jl

In [47]:
@var x y z
TwistedCubic=System([y^2-x*z,x*y-z,x^2-y])

System of length 3
 3 variables: x, y, z

 -x*z + y^2
 -z + x*y
 -y + x^2

In [177]:
#Try the following without codimension: you get an error because it expects the dimension to be zero.
W_twisted=witness_set(TwistedCubic; codim=2)

Witness set for dimension 1 of degree 3

In [178]:
#Degree of the twisted cubic
degree(W_twisted)

3

In [179]:
#Dimension of the twisted cubic
dim(W_twisted)

1

In [180]:
#Generic linear space used to slice the twisted cubic
W_twisted.L

2-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.1829742479854597 + 0.7548882493402352im 0.24815572302119737 - 0.44967207103529405im 0.3511647255505307 + 0.09778167913196648im]
b:
Complex{Float64}[0.23506515634712388 + 0.25175255380489947im]

In [181]:
#Witness points
solutions(W_twisted)

3-element Array{Array{Complex{Float64},1},1}:
 [-1.2318922827570007 + 1.9750827058453988im, -2.383393098613327 - 4.866178286275525im, 12.547188141741241 + 1.2872089870800463im]
 [0.8378937395228422 - 0.13041748804292835im, 0.6850571975441452 - 0.21855199351092955im, 0.5455021350368323 - 0.27246678599248036im]
 [0.06908492478284081 - 0.4736770883269493im, -0.21959725717364575 - 0.06544789203684467im, -0.04617202693150863 + 0.09949672668403586im]

## Some simple algorithms involving witness sets

### Moving the linear space (this is the fundamental operation)

In [182]:
L2 = LinearSubspace([1,2,3]',[5])

2-dim. affine linear subspace {x | Ax=b} with eltype Float64:
A:
[-0.2672612419124243 -0.5345224838248488 -0.8017837257372732]
b:
[-1.3363062095621219]

In [183]:
W_twisted2=witness_set(W_twisted,L2)

Witness set for dimension 1 of degree 3

In [184]:
solutions(W_twisted2)

3-element Array{Array{Complex{Float64},1},1}:
 [0.9241438381955056 - 1.793662034335766e-43im, 0.8540418336747209 - 1.3452465257518244e-43im, 0.7892574981516842 + 0.0im]
 [-0.7954052524310862 - 1.0820358328370792im, -0.5381320279484715 + 1.7213139695145148im, 2.290556436109343 - 0.786864035397317im]
 [-0.7954052524310862 + 1.0820358328370792im, -0.5381320279484714 - 1.721313969514515im, 2.290556436109343 + 0.7868640353973171im]

### Sampling

In [185]:
function sample(W::WitnessSet)
    n=length(solutions(W)[1]) #number of variables
    randomLinear = LinearSubspace(randn(ComplexF64,dim(W),n),randn(ComplexF64,dim(W)))
    newWitness=witness_set(W,randomLinear)
    s=solutions(newWitness)[1]
    return s
end

sample (generic function with 1 method)

In [186]:
MySamples=[sample(W_twisted) for i in 1:100]

100-element Array{Array{Complex{Float64},1},1}:
 [0.30092071057189734 + 0.9811339155709625im, -0.8720704862325129 + 0.5904870300796039im, -0.8417709223017383 - 0.6779281541361348im]
 [0.5283859660628376 - 0.836636420428293im, -0.4207687708549093 - 0.884133886502716im, -0.962026923460231 - 0.11513345947256473im]
 [-0.5526892557128238 + 0.36316100467021534im, 0.17357949806731499 - 0.40143037075020527im, 0.04984833315295171 + 0.2849035577387317im]
 [-0.671315087394244 + 0.4355727688561184im, 0.2609403095941558 - 0.5848131427823958im, 0.0795555131252949 + 0.5062523792123658im]
 [-0.6443992898101225 + 1.611891627285316im, -2.1829441734047146 - 2.077403639747081im, 4.755237208437503 - 2.179992005840342im]
 [-0.7586795016191666 - 0.01582189205117807im, 0.575344253909028 + 0.02400749035212006im, -0.43612204789438175 - 0.027317025493087628im]
 [-0.9636290636586927 - 0.8246387820105335im, 0.24855185153191242 + 1.5892917947309104im, 1.0710798619039017 - 1.7364532601507296im]
 [-4.75678809265784 -

### Membership

In [187]:
function membership(W::WitnessSet,p)
    n=length(solutions(W)[1]) #number of variables
    A=randn(ComplexF64,dim(W),n)
    randomLinearThroughP = LinearSubspace(A,A*p)
    newWitness=witness_set(W,randomLinearThroughP)
    check=findfirst(x->x≈p,solutions(newWitness))
    if check==nothing
        return false
    else
        return true
    end
end
    

membership (generic function with 1 method)

In [188]:
membership(W_twisted,[2,4,8])

true

In [189]:
membership(W_twisted,[2,5,8])

false

In [190]:
[membership(W_twisted,s) for s in MySamples]

100-element Array{Bool,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

## Pseudo witness sets

Both the sampling and membership algorithms (and most other algorithms involving witness sets) depend on one thing:


**the ability to move the linear space of the witness set and track the witness points**

Observation: we can do this for projections trivially

**Definition:** A pseudo witness set for an (irreducible) variety $X\subset \mathbb{C}^n$ is a quadruple 
$$(F,\pi,\pi^{-1}(L),S)$$
where 
- $\pi:\mathbb{C}^N \to \mathbb{C}^n$ (really this works with any map by factoring through the graph)
- $X$ is a component of $\pi(\mathcal{V}(F))$ of the same dimension as $\mathcal{V}(F)$
- $L$ is a linear space of complementary dimension (to $X$) in $\mathbb{C}^n$
- $S = \pi^{-1}(L) \cap \mathcal V(F)$ (note! this is in $\mathbb{C}^N$)

<img src="PseudoWitness1.png" width="300">

As the picture above shows, $\pi^{-1}(L)$ may not be generic for $\mathcal{V}(F)$ even though $L$ is generic for $X$


**Warning** be careful about the following situation

<img src="Careful.png" width="200">

In [62]:
PseudoWitness_Parabola=witness_set(W_twisted,LinearSubspace([3,6,0]',[1]))

Witness set for dimension 1 of degree 2

In [63]:
ProjectedSolutions=[s[1:2] for s in solutions(PseudoWitness_Parabola)]

2-element Array{Array{Complex{Float64},1},1}:
 [-0.728713553878169 + 0.0im, 0.5310234436057513 + 0.0im]
 [0.22871355387816908 - 7.52316384526264e-37im, 0.05230988972758215 + 0.0im]

In [64]:
Witness_Parabola=witness_set(System([y-x^2])) #computes a witness set for the parabola
Witness_Parabola2=witness_set(Witness_Parabola,LinearSubspace([3,6]',[1])) #moves it to the space we used above

Witness set for dimension 1 of degree 2

In [65]:
solutions(Witness_Parabola2)

2-element Array{Array{Complex{Float64},1},1}:
 [0.22871355387816908 + 3.587324068671532e-43im, 0.05230988972758215 + 8.96831017167883e-44im]
 [-0.728713553878169 + 9.62964972193618e-35im, 0.5310234436057513 - 9.62964972193618e-35im]

Membership/sampling can be easily coded for these pseudo-witness sets as well

## Witness supersets, monodromy, the trace test, and irreducible decomposition

In [191]:
#If we only take two of the generators, we get a complete intersection of the union of a cubic and a line
CompleteIntersection=System([y^2-x*z,x*y-z])
W_complete=witness_set(CompleteIntersection)

Witness set for dimension 1 of degree 4


<img src="witnesscubic.png" width="200">

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Warning: W_complete is not a witness set for the twisted cubic.
</p>

A **witness superset** for $\mathcal{V}(f_1,\ldots,f_k)$ in codimension $m$ is the triple $W=(F,L,S)$
    
- $\color{blue}{F}$ is a polynomial system
- $\color{green}{L}$ is a generic affine linear space of dimension $m$
- $\color{red}{S} = X \cap L$


Let $X \subseteq \mathbb{C}^n$ be an (equidimensional) variety of codimension $m$.

We have a branched cover 


$$Z = \{(x,L) \in \mathbb{C}^n \times \textrm{Gr}(m+1,n+1) \mid x \in X \cap L \} \subset \mathbb{C}^n \times \textrm{Gr}(m+1,n+1)$$
$$\,\,\,\, \downarrow \pi $$
$$\textrm{Gr}(m+1,n+1)$$



**Theorem**: If $X$ is irreducible then the monodromy group of $\pi$ is the full symmetric group.

**Corollary**: If $X=X_1,\ldots,X_k$ is equidimensional, then the monodromy group of $\pi$  is the product of symmetric groups $\mathfrak{S}_{\textrm{deg}(X_1)} \times \cdots \times \mathfrak{S}_{\textrm{deg}(X_k)}$

Let's try to compute the witness points using monodromy

In [200]:
M=monodromy_solve(CompleteIntersection; dim=1) 

MonodromyResult
• return_code → :success
• 3 solutions
• 15 tracked loops
• random_seed → 0x4aa54664
• trace → 3.0808358375128905e-17

In [201]:
#Trying this many times will give us either 1 or 3 solutions (possibly 2 if the heuristic stop didn't work)
[length(solutions(monodromy_solve(CompleteIntersection;dim=1))) for i in 1:5]

5-element Array{Int64,1}:
 3
 1
 3
 3
 3

**Monodromy**: allows us to find points on the same component

**The trace test**: tells us when we've found all points on the same component


The **trace** of a subset of points in $\mathbb{C}^n$ is their coordinate-wise average.

**Lemma**: Let $X$ be irreducible. The trace of a subset of witness points of $X$ moves linearly if and only if the subset is not proper.


<img src="TraceExample1.png" width="600">

Now let's try moving a witness set and verifying this ourselves

In [202]:
W_complete

Witness set for dimension 1 of degree 4

In [203]:
A=randn(ComplexF64,1,3)
b=rand(ComplexF64)
L₁ = LinearSubspace(A,[b])
L₂ = LinearSubspace(A,[b+1])
L₃ = LinearSubspace(A,[b+2]);

In [204]:
trace₁=sum(solutions(witness_set(W_complete,L₁)))
println("First trace: \n",trace₁)
trace₂=sum(solutions(witness_set(W_complete,L₂)))
println("Second trace: \n",trace₂)
trace₃=sum(solutions(witness_set(W_complete,L₃)))
println("Third trace: \n",trace₃)

First trace: 
Complex{Float64}[-5.843550581078897 - 4.837226691475427im, -0.866988403226279 + 11.525374911525777im, 36.459481897813376 - 18.168165454802896im]
Second trace: 
Complex{Float64}[-9.945229992090425 - 5.551123769488707im, -0.8669884032262762 + 11.525374911525775im, 43.66544106293409 - 14.413232593364057im]
Third trace: 
Complex{Float64}[-14.046909403101957 - 6.2650208475019875im, -0.8669884032262789 + 11.525374911525777im, 50.87140022805485 - 10.658299731925203im]


In [205]:
# here we check trace2 is the midpoint of trace1 and trace3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 -3.552713678800501e-15 - 1.7763568394002505e-15im
 -5.329070518200751e-15 + 3.552713678800501e-15im
  4.263256414560601e-14 + 1.4210854715202004e-14im

In [210]:
trace_test(W_complete)

3.8799593666382355e-17

We can find out which solution is the one on the line by checking traces of subsets

In [216]:
I=[1,3,4]
trace₁=sum(solutions(witness_set(W_complete,L₁))[I])
trace₂=sum(solutions(witness_set(W_complete,L₂))[I])
trace₃=sum(solutions(witness_set(W_complete,L₃))[I])
# here we check trace2 is the midpoint of trace1 and trace 3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
                    0.0 - 8.881784197001252e-16im
 -5.329070518200751e-15 + 3.552713678800501e-15im
  4.263256414560601e-14 + 1.4210854715202004e-14im

In [217]:
[membership(W_twisted,s) for s in solutions(W_complete)]

4-element Array{Bool,1}:
 1
 0
 1
 1

### Numerical irreducible decomposition (of equidimensional varieties)

One of the most important algorithms in numerical algebraic geometry is taking a polynomial systme $F$ and computing a witness set for every irreducible component of $\mathcal V(F)$. This is called a **numerical irreducible decomposition**. We will go through how it works for equidimensional varieties

#### Step 1: compute a witness superset 

$\mathcal V(F) \to (F,L,S)$

#### Step 2: 

Using monodromy, identify solutions as belonging to the same component until the trace test passes.

$p \in S \to S_p=\{q \in S \mid q \text{ was found via monodromy starting from } p\}$

#### Step 3: 

If $S \backslash \bigcup_{p \text{ done in step 2}} S_p \neq \emptyset$ pick some $q$ not identified and repeat step 2

#### Output: 

Set $W_p = (F,L,S_p)$ and output the union of all $W_p$

### Other things you can do with witness sets

Given a witness set $W$ for $X$ you can also...

- find a witness set for $X \cap \mathcal V(f)$
- recover equations for $X$
- compute the Newton polytope of $X$ (when $X$ is a hypersurface)
- describe the real points of $X$
- much more...